In [33]:
import numpy as np
import pandas as pd
import ast

In [46]:
# Lectura del dataset de creditos
credits = pd.read_csv('../Datasets/credits.csv', low_memory=False)
credits.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [35]:
# Verificamos la cantidad de regitros nulos por columna
credits.isna().sum()

cast    0
crew    0
id      0
dtype: int64

# Desanidamos las columnas cast, crew, id y la volvemos un dataframe
* Columna cast

In [36]:
# Asegurarnos de que la columna cast sea interpretada correctamente como lista de diccionarios
credits['cast'] = credits['cast'].apply(lambda x: ast.literal_eval(x))

# Convertir cada elemento de la columna 'cast' en un DataFrame
casting = credits['cast'].apply(pd.DataFrame)

# Asignar un índice para cada fila en el DataFrame original
credits['index'] = credits.index

# Combinar todos los DataFrames individuales en uno solo
#casting_expanded = pd.concat([df.assign(index=idx) for idx, df in casting.items()])

# Añadir la columna 'id' al DataFrame desanidado y combinar todos los DataFrames individuales en uno solo
casting_expanded = pd.concat([df.assign(id=credits.loc[idx, 'id']) for idx, df in casting.items()])

In [40]:
# Eliminamos la columna 'index'
casting_expanded = casting_expanded.drop(columns=['credit_id', 'gender', 'cast_id', 'profile_path'])	

*Una lista de diccionarios que representa los actores del reparto de la película. Cada diccionario contiene información detallada sobre cada miembro del reparto.*

In [42]:
casting_expanded.head()

,character,id,name,order
0,Woody (voice),862,Tom Hanks,0.0
1,Buzz Lightyear (voice),862,Tim Allen,1.0
2,Mr. Potato Head (voice),862,Don Rickles,2.0
3,Slinky Dog (voice),862,Jim Varney,3.0
4,Rex (voice),862,Wallace Shawn,4.0


* Columna crew

In [47]:
# Asegurarnos de que la columna cast sea interpretada correctamente como lista de diccionarios
credits['crew'] = credits['crew'].apply(lambda x: ast.literal_eval(x))

# Convertir cada elemento de la columna 'crew' en un DataFrame
crew_1 = credits['crew'].apply(pd.DataFrame)

# Asignar un índice para cada fila en el DataFrame original
credits['index'] = credits.index

# Combinar todos los DataFrames individuales en uno solo
#crew_expanded = pd.concat([df.assign(index=idx) for idx, df in crew_1.items()])

# Añadir la columna 'id' al DataFrame desanidado y combinar todos los DataFrames individuales en uno solo
crew_expanded = pd.concat([df.assign(id=credits.loc[idx, 'id']) for idx, df in crew_1.items()])

In [49]:
# Eliminamos la columna 'index'
crew_expanded = crew_expanded.drop(columns=['credit_id', 'gender', 'profile_path'])	

*Una lista de diccionarios que representa los miembros del equipo de producción de la película. Cada diccionario contiene información detallada sobre cada miembro del equipo.*

In [93]:
crew_expanded.head()

,department,id,job,name
0,Directing,862,Director,John Lasseter
1,Writing,862,Screenplay,Joss Whedon
2,Writing,862,Screenplay,Andrew Stanton
3,Writing,862,Screenplay,Joel Cohen
4,Writing,862,Screenplay,Alec Sokolow


* Columna id

In [13]:
col_id = credits['id']
col_id.head()

0      862
1     8844
2    15602
3    31357
4    11862
Name: id, dtype: int64

* Verificamos si hay valores nulos de cada dataframe

In [14]:
print(casting_expanded.isna().sum())
print(crew_expanded.isna().sum())
print(col_id.isna().sum())

cast_id              0
character            0
credit_id            0
gender               0
id                   0
name                 0
order                0
profile_path    173856
dtype: int64
credit_id            0
department           0
gender               0
id                   0
job                  0
name                 0
profile_path    369216
dtype: int64
0


### Exportamos los datasets a csv

In [51]:
casting_expanded.to_csv('../Datasets/casting_expanded.csv', index=True)

In [139]:
casting_expanded.to_parquet('../Datasets/casting_expanded.parquet', index=True)

In [137]:
crew_expanded.to_csv('../Datasets/crew_expanded.csv', index=True)

In [138]:
crew_expanded.to_parquet('../Datasets/crew_expanded.parquet', index=True)

In [53]:
col_id.to_csv('../Datasets/col_id.csv', index=True)

## Vamos a crear un dataframe para los endpoints 5 y 6

In [111]:
# Cargamos el dataset de peliculas
movies_2 = pd.read_csv('../Datasets/movies.csv', low_memory=False)

In [112]:
# Asegurarse de que la columna 'id' sea del mismo tipo en todos los DataFrames
movies_2['id'] = movies_2['id'].astype(str)
casting_expanded['id'] = casting_expanded['id'].astype(str)
crew_expanded['id'] = crew_expanded['id'].astype(str)

In [118]:
# Realizamos la union de los df
union_mcasting = pd.merge(movies_2, casting_expanded, on='id', how='inner')

In [129]:
# Realizamos la union de los df
union_mcrew = pd.merge(movies_2, crew_expanded, on='id', how='inner')

In [123]:
# Eliminamos columnas que no vamos a usar
union_mcasting = union_mcasting.drop(columns=['original_language', 'overview', 'runtime', 'tagline', 'vote_average', 'vote_count', 'status', 'popularity', 'order'])

In [130]:
# Eliminamos columnas que no vamos a usar
union_mcrew = union_mcrew.drop(columns=['original_language', 'overview', 'runtime', 'tagline', 'vote_average', 'vote_count', 'status', 'popularity', 'department'])

In [144]:
union_mcasting.head()

,budget,id,release_date,revenue,title,release_year,return,character,name
0,30000000.0,862,1995-10-30,373554033.0,Toy Story,1995.0,12.451801,Woody (voice),Tom Hanks
1,30000000.0,862,1995-10-30,373554033.0,Toy Story,1995.0,12.451801,Buzz Lightyear (voice),Tim Allen
2,30000000.0,862,1995-10-30,373554033.0,Toy Story,1995.0,12.451801,Mr. Potato Head (voice),Don Rickles
3,30000000.0,862,1995-10-30,373554033.0,Toy Story,1995.0,12.451801,Slinky Dog (voice),Jim Varney
4,30000000.0,862,1995-10-30,373554033.0,Toy Story,1995.0,12.451801,Rex (voice),Wallace Shawn


In [146]:
union_mcrew.head()

,budget,id,release_date,revenue,title,release_year,return,job,name
0,30000000.0,862,1995-10-30,373554033.0,Toy Story,1995.0,12.451801,Director,John Lasseter
1,30000000.0,862,1995-10-30,373554033.0,Toy Story,1995.0,12.451801,Screenplay,Joss Whedon
2,30000000.0,862,1995-10-30,373554033.0,Toy Story,1995.0,12.451801,Screenplay,Andrew Stanton
3,30000000.0,862,1995-10-30,373554033.0,Toy Story,1995.0,12.451801,Screenplay,Joel Cohen
4,30000000.0,862,1995-10-30,373554033.0,Toy Story,1995.0,12.451801,Screenplay,Alec Sokolow


## Exportamos los dfs nuevos

In [134]:
union_mcasting.to_parquet('../Datasets/union_mcasting.parquet', index=False)

In [135]:
union_mcrew.to_parquet('../Datasets/union_mcrew.parquet', index=False)